## question answer - GAN

In [1]:
## train GAN on this two dataset

In [2]:
'''import csv
with open('first_gan.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['question_id','question','target_answer'])
    for i in range(0,len(Question_id)):
        writer.writerow([Question_id[i],Question[i],Target_answer[i]])'''

'import csv\nwith open(\'first_gan.csv\', \'w\') as datacsv:\n    writer = csv.writer(datacsv,dialect=("excel"))\n    writer.writerow([\'question_id\',\'question\',\'target_answer\'])\n    for i in range(0,len(Question_id)):\n        writer.writerow([Question_id[i],Question[i],Target_answer[i]])'

In [3]:
'''import csv
with open('second_gan.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['question_id','question','candidate_answer','target_answer'])
    for i in range(0,len(Question_id)):
        writer.writerow([Question_id[i],Question[i],Document[i],Target_answer[i]])'''

'import csv\nwith open(\'second_gan.csv\', \'w\') as datacsv:\n    writer = csv.writer(datacsv,dialect=("excel"))\n    writer.writerow([\'question_id\',\'question\',\'candidate_answer\',\'target_answer\'])\n    for i in range(0,len(Question_id)):\n        writer.writerow([Question_id[i],Question[i],Document[i],Target_answer[i]])'

## generator

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import json
from collections import defaultdict
import codecs
from sklearn.utils import shuffle
from collections import Counter
import itertools
import re
import math
import os
import random
import sys
import time
from six.moves import xrange  # pylint: disable=redefined-builtin

In [2]:
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import ops
from tensorflow.python.util import nest
from tensorflow.python.ops import variable_scope

In [3]:
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

In [4]:
def softmax(x):
    prob = np.exp(x) / np.sum(np.exp(x), axis=0)
    return prob

In [5]:
def sequence_loss_by_example(logits, targets, weights,
                             average_across_timesteps=True,
                             softmax_loss_function=None, name=None):
    # print('sequence_loss_by_example')
    if len(targets) != len(logits) or len(weights) != len(logits):
        raise ValueError("Lengths of logits, weights, and targets must be the same "
                     "%d, %d, %d." % (len(logits), len(weights), len(targets)))
    with ops.name_scope(name, "sequence_loss_by_example",
                      logits + targets + weights):
        log_perp_list = []
        for logit, target, weight in zip(logits, targets, weights):
            if softmax_loss_function is None:
                target = array_ops.reshape(target, [-1])
                crossent = nn_ops.sparse_softmax_cross_entropy_with_logits(logit, target)
            else:
                crossent = softmax_loss_function(logit, target)
            log_perp_list.append(crossent * weight)
        log_perps = math_ops.add_n(log_perp_list)
        if average_across_timesteps:
            total_size = math_ops.add_n(weights)
            total_size += 1e-12  # Just to avoid division by 0 for all-0 weights.
            log_perps /= total_size
    return log_perps

In [6]:
def sequence_loss(logits, targets, weights,
                  average_across_timesteps=True, average_across_batch=True,
                  softmax_loss_function=None, name=None):
    #print("sequence_loss")
    with ops.name_scope(name, "sequence_loss", logits + targets + weights):
        cost = math_ops.reduce_sum(sequence_loss_by_example(
                                                        logits, targets, weights,
                                                        average_across_timesteps=average_across_timesteps,
                                                        softmax_loss_function=softmax_loss_function))
        if average_across_batch:
            batch_size = array_ops.shape(targets[0])[0]
            return cost / math_ops.cast(batch_size, cost.dtype)
        else:
            return cost

In [7]:
_BIAS_VARIABLE_NAME = "bias"
_WEIGHTS_VARIABLE_NAME = "kernel"
def linear(args,
            output_size,
            bias,
            bias_initializer=None,
            kernel_initializer=None):
    if args is None or (nest.is_sequence(args) and not args):
        raise ValueError("`args` must be specified")
    if not nest.is_sequence(args):
        args = [args]

    # Calculate the total size of arguments on dimension 1.
    total_arg_size = 0
    shapes = [a.get_shape() for a in args]
    for shape in shapes:
        if shape.ndims != 2:
            raise ValueError("linear is expecting 2D arguments: %s" % shapes)
        if shape[1].value is None:
            raise ValueError("linear expects shape[1] to be provided for shape %s, "
                       "but saw %s" % (shape, shape[1]))
        else:
            total_arg_size += shape[1].value

    dtype = [a.dtype for a in args][0]
    
    # Now the computation.
    scope = vs.get_variable_scope()
    with vs.variable_scope(scope) as outer_scope:
        weights = vs.get_variable(
                                    _WEIGHTS_VARIABLE_NAME, [total_arg_size, output_size],
                                    dtype=dtype,
                                    initializer=kernel_initializer)
        if len(args) == 1:
            res = math_ops.matmul(args[0], weights)
        else:
            res = math_ops.matmul(array_ops.concat(args, 1), weights)
        if not bias:
            return res
        with vs.variable_scope(outer_scope) as inner_scope:
            inner_scope.set_partitioner(None)
            if bias_initializer is None:
                bias_initializer = init_ops.constant_initializer(0.0, dtype=dtype)
            biases = vs.get_variable(
                                      _BIAS_VARIABLE_NAME, [output_size],
                                      dtype=dtype,
                                        initializer=bias_initializer)
        return nn_ops.bias_add(res, biases)

In [8]:
def attention_decoder(decoder_inputs,
                      initial_state,
                      attention_states,
                      cell,
                      output_size=None,
                      num_heads=1,
                      loop_function=None,
                      dtype=None,
                      scope=None,
                      initial_state_attention=False):

    #print('attention_decoder')
    if not decoder_inputs:
        raise ValueError("Must provide at least 1 input to attention decoder.")
    if num_heads < 1:
        raise ValueError("With less than 1 heads, use a non-attention decoder.")
    if attention_states.get_shape()[2].value is None:
        raise ValueError("Shape[2] of attention_states must be known: %s"
                     % attention_states.get_shape())
    if output_size is None:
        output_size = cell.output_size

    with variable_scope.variable_scope(scope or "attention_decoder", dtype=dtype) as scope:
        dtype = scope.dtype

        batch_size = array_ops.shape(decoder_inputs[0])[0]  # Needed for reshaping.
        attn_length = attention_states.get_shape()[1].value
        if attn_length is None:
            attn_length = shape(attention_states)[1]
        attn_size = attention_states.get_shape()[2].value

        # To calculate W1 * h_t we use a 1-by-1 convolution, need to reshape before.
        hidden = array_ops.reshape(attention_states, [-1, attn_length, 1, attn_size])
        hidden_features = []
        v = []
        attention_vec_size = attn_size  # Size of query vectors for attention.
        for a in xrange(num_heads):
            k = variable_scope.get_variable("AttnW_%d" % a,
                                      [1, 1, attn_size, attention_vec_size])
            hidden_features.append(nn_ops.conv2d(hidden, k, [1, 1, 1, 1], "SAME"))
            v.append(variable_scope.get_variable("AttnV_%d" % a, [attention_vec_size]))

        state = initial_state

        def attention(query):
            """Put attention masks on hidden using hidden_features and query."""
            #print("attention_decoder.attention")
            ds = []  # Results of attention reads will be stored here.
            if nest.is_sequence(query):  # If the query is a tuple, flatten it.
                query_list = nest.flatten(query)
                for q in query_list:  # Check that ndims == 2 if specified.
                    ndims = q.get_shape().ndims
                    if ndims:
                        assert ndims == 2
                query = array_ops.concat(query_list,1)
            for a in xrange(num_heads):
                with variable_scope.variable_scope("Attention_%d" % a):
                    #print('调用linear')
                    y = linear(query, attention_vec_size,True)
                    y = array_ops.reshape(y, [-1, 1, 1, attention_vec_size])
                    # Attention mask is a softmax of v^T * tanh(...).
                    s = math_ops.reduce_sum(v[a] * math_ops.tanh(hidden_features[a] + y), [2, 3])
                    a = nn_ops.softmax(s)
                    # Now calculate the attention-weighted vector d.
                    d = math_ops.reduce_sum(
                    array_ops.reshape(a, [-1, attn_length, 1, 1]) * hidden,[1, 2])
                    ds.append(array_ops.reshape(d, [-1, attn_size]))
            return ds

        outputs = []
        prev = None
        batch_attn_size = array_ops.stack([batch_size, attn_size])
        attns = [array_ops.zeros(batch_attn_size, dtype=dtype)for _ in xrange(num_heads)]
        for a in attns:  # Ensure the second shape of attention vectors is set.
            a.set_shape([None, attn_size])
        if initial_state_attention:
            attns = attention(initial_state)
        for i, inp in enumerate(decoder_inputs):
            if i > 0:
                variable_scope.get_variable_scope().reuse_variables()
            # If loop_function is set, we use it instead of decoder_inputs.
            if loop_function is not None and prev is not None:
                with variable_scope.variable_scope("loop_function", reuse=True):
                    inp = loop_function(prev, i)
            # Merge input and previous attentions into one vector of the right size.
            input_size = inp.get_shape().with_rank(2)[1]
            if input_size.value is None:
                raise ValueError("Could not infer input size from input: %s" % inp.name)
                
            #print('调用linear')
            x = linear([inp] + attns, input_size,True)
            cell_output, state = cell(x, state)
            if i == 0 and initial_state_attention:
                #print('找错1')
                with variable_scope.variable_scope(variable_scope.get_variable_scope(),
                                           reuse=True):
                    #print("找错2")
                    attns = attention(state)
            else:
                #print("找错3")
                attns = attention(state)
            
            with variable_scope.variable_scope("AttnOutputProjection"):
                #print('output = linear([cell_output] + attns, output_size,True)')
                output = linear([cell_output] + attns, output_size,True)
            if loop_function is not None:
                prev = output
            #print('outputs.append(output)')
            outputs.append(output)

    return outputs, state

In [9]:
def _argmax_or_mcsearch(embedding, output_projection=None, update_embedding=True, mc_search=False):
    def loop_function(prev, _):
        if output_projection is not None:
            prev = nn_ops.xw_plus_b(prev, output_projection[0], output_projection[1])


        if isinstance(mc_search, bool):
            prev_symbol = tf.reshape(tf.multinomial(prev, 1), [-1]) if mc_search else math_ops.argmax(prev, 1)
        else:
            prev_symbol = tf.cond(mc_search, lambda: tf.reshape(tf.multinomial(prev, 1), [-1]), lambda: tf.argmax(prev, 1))


        emb_prev = embedding_ops.embedding_lookup(embedding, prev_symbol)
        if not update_embedding:
            emb_prev = array_ops.stop_gradient(emb_prev)
        return emb_prev
    return loop_function

In [10]:
def embedding_attention_decoder(decoder_inputs,
                                initial_state,
                                attention_states,
                                cell,
                                num_symbols,
                                embedding_size,
                                num_heads=1,
                                output_size=None,
                                output_projection=None,
                                feed_previous=False,
                                update_embedding_for_previous=True,
                                dtype=None,
                                scope=None,
                                initial_state_attention=False,
                                mc_search = False):
    #print('embedding_attention_decoder')
    if output_size is None:
        output_size = cell.output_size
    if output_projection is not None:
        proj_biases = ops.convert_to_tensor(output_projection[1], dtype=dtype)
        proj_biases.get_shape().assert_is_compatible_with([num_symbols])

    with variable_scope.variable_scope(scope or "embedding_attention_decoder", dtype=dtype) as scope:
        embedding = variable_scope.get_variable("embedding",[num_symbols, embedding_size])

        loop_function = None
        if feed_previous == True:
            loop_function = _argmax_or_mcsearch(embedding, output_projection, update_embedding_for_previous, mc_search)

        emb_inp = [embedding_ops.embedding_lookup(embedding, i) for i in decoder_inputs]
        return attention_decoder(
                                emb_inp,
                                initial_state,
                                attention_states,
                                cell,
                                output_size=output_size,
                                num_heads=num_heads,
                                loop_function=loop_function,
                                initial_state_attention=initial_state_attention,
                                scope=scope)




In [11]:
# return output, state, encoder_state

def embedding_attention_seq2seq(encoder_inputs,
                                decoder_inputs,
                                cell,
                                num_encoder_symbols,
                                num_decoder_symbols,
                                embedding_size,
                                num_heads=1,
                                output_projection=None,
                                feed_previous=False,
                                dtype=None,
                                scope=None,
                                initial_state_attention=False,
                                mc_search=False):

    with variable_scope.variable_scope(scope or "embedding_attention_seq2seq", dtype=dtype) as scope:
        dtype = scope.dtype
        #print('embedding_attention_seq2seq')
        
        # Encoder.
        #print('encoder_cell')
        encoder_cell = tf.contrib.rnn.core_rnn_cell.EmbeddingWrapper(
                cell, embedding_classes=num_encoder_symbols,
                embedding_size=embedding_size)
        #print('encoder_outputs, encoder_state = tf.contrib.rnn.static_rnn')
        encoder_outputs, encoder_state = tf.contrib.rnn.static_rnn(
                encoder_cell, encoder_inputs, dtype=dtype)
        #print('encoder_outputs')
        
        # First calculate a concatenation of encoder outputs to put attention on.
        #print('top-state')
        #top_states = [array_ops.reshape([-1, 1, cell.output_size],e)for e in encoder_outputs]
        top_states = tf.stack(encoder_outputs)
        top_states = tf.transpose(top_states, [1,0,2])
        #print('attention_state')
        attention_states = array_ops.concat(top_states,1)

        # Decoder.
        #print('decoder')
        output_size = None
        if output_projection is None:
            cell = rnn_cell.OutputProjectionWrapper(cell, num_decoder_symbols)
            output_size = num_decoder_symbols

        if isinstance(feed_previous, bool):
            outputs, state = embedding_attention_decoder(
                          decoder_inputs,
                          encoder_state,
                          attention_states,
                          cell,
                          num_decoder_symbols,
                          embedding_size,
                          num_heads=num_heads,
                          output_size=output_size,
                          output_projection=output_projection,
                          feed_previous=feed_previous,
                          initial_state_attention=initial_state_attention,
                          mc_search=mc_search,
                          scope=scope)
            return outputs, state, encoder_state

        # If feed_previous is a Tensor, we construct 2 graphs and use cond.
        def decoder(feed_previous_bool):
            reuse = None if feed_previous_bool else True
            with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=reuse) as scope:
                outputs, state = embedding_attention_decoder(
                            decoder_inputs,
                            encoder_state,
                            attention_states,
                            cell,
                            num_decoder_symbols,
                            embedding_size,
                            num_heads=num_heads,
                            output_size=output_size,
                            output_projection=output_projection,
                            feed_previous=feed_previous_bool,
                            update_embedding_for_previous=False,
                            initial_state_attention=initial_state_attention,
                            mc_search=mc_search,
                            scope=scope)
                state_list = [state]
                if nest.is_sequence(state):
                    state_list = nest.flatten(state)
                return outputs + state_list

        outputs_and_state = control_flow_ops.cond(feed_previous,
                                              lambda: decoder(True),
                                              lambda: decoder(False))
        outputs_len = len(decoder_inputs)  # Outputs length same as decoder inputs.
        state_list = outputs_and_state[outputs_len:]
        state = state_list[0]
        if nest.is_sequence(encoder_state):
            state = nest.pack_sequence_as(structure=encoder_state,
                                    flat_sequence=state_list)
        return outputs_and_state[:outputs_len], state, encoder_state





In [12]:
# return outputs, losses, encoder_states

def model_with_buckets(encoder_inputs, decoder_inputs, targets, weights, buckets, vocab_size, batch_size, seq2seq,
                       output_projection=None, softmax_loss_function=None, per_example_loss=False, name=None):
    #print('model_with_buckets')
    if len(encoder_inputs) < buckets[-1][0]:
        raise ValueError("Length of encoder_inputs (%d) must be at least that of la"
                     "st bucket (%d)." % (len(encoder_inputs), buckets[-1][0]))
    if len(targets) < buckets[-1][1]:
        raise ValueError("Length of targets (%d) must be at least that of last"
                     "bucket (%d)." % (len(targets), buckets[-1][1]))
    if len(weights) < buckets[-1][1]:
        raise ValueError("Length of weights (%d) must be at least that of last"
                     "bucket (%d)." % (len(weights), buckets[-1][1]))

    all_inputs = encoder_inputs + decoder_inputs + targets + weights
    losses = []
    outputs = []
    encoder_states = []
    with ops.name_scope(name, "model_with_buckets", all_inputs):
        for j, bucket in enumerate(buckets):
            with variable_scope.variable_scope(variable_scope.get_variable_scope(),
                                         reuse=True if j > 0 else None):
                bucket_outputs, decoder_states, encoder_state = seq2seq(encoder_inputs[:bucket[0]],
                                    decoder_inputs[:bucket[1]])
                outputs.append(bucket_outputs)
                #print("bucket outputs: %s" %bucket_outputs)
                encoder_states.append(encoder_state)
                if per_example_loss:
                    losses.append(sequence_loss_by_example(
                    outputs[-1], targets[:bucket[1]], weights[:bucket[1]],
                    softmax_loss_function=softmax_loss_function))
                else:
                    # losses.append(sequence_loss_by_mle(outputs[-1], targets[:bucket[1]], vocab_size, bucket[1], batch_size, output_projection))
                    losses.append(sequence_loss(outputs[-1], targets[:bucket[1]], weights[:bucket[1]], softmax_loss_function=softmax_loss_function))

    return outputs, losses, encoder_states

In [13]:
class Seq2SeqModel(object):
    def __init__(self, config,name_scope, forward_only,use_lstm=False,num_samples=512):

        dtype=tf.float32
        
        source_vocab_size = config.vocab_size
        target_vocab_size = config.vocab_size
        size= config.emb_dim

        self.buckets = config.buckets
        self.batch_size = config.batch_size
        self.learning_rate = tf.Variable(float(config.learning_rate), trainable=False)
        self.learning_rate_decay_op = self.learning_rate.assign(
            self.learning_rate * config.learning_rate_decay_factor)
        self.global_step = tf.Variable(0, trainable=False)
        max_gradient_norm = config.max_gradient_norm
        self.forward_only = tf.placeholder(tf.bool, name="forward_only")
        #self.num_layers = config.num_layers
        num_layers = config.num_layers
        self.max_gradient_norm = config.max_gradient_norm
        
        # If we use sampled softmax, we need an output projection.
        output_projection = None
        softmax_loss_function = None
        
        # ADD
        self.mc_search = tf.placeholder(tf.bool, name="mc_search")
        self.up_reward = tf.placeholder(tf.bool, name="up_reward")
        self.reward_bias = tf.get_variable("reward_bias", [1], dtype=tf.float32)
        
        # Sampled softmax only makes sense if we sample less than vocabulary size.
        if num_samples > 0 and num_samples < target_vocab_size:
            w = tf.get_variable("proj_w", [size, target_vocab_size])
            w_t = tf.transpose(w)
            b = tf.get_variable("proj_b", [target_vocab_size])
            output_projection = (w, b)

            def sampled_loss(inputs, labels):
                
                labels = tf.reshape(labels, [-1, 1])
                local_w_t = tf.cast(w_t, tf.float32)
                local_b = tf.cast(b, tf.float32)
                local_inputs = tf.cast(inputs, tf.float32)
                
                return tf.cast(
                    tf.nn.sampled_softmax_loss(local_w_t, local_b, labels, local_inputs,
                                               num_samples, target_vocab_size), dtype)
            softmax_loss_function = sampled_loss

        # Create the internal multi-layer cell for our RNN.
        single_cell = tf.contrib.rnn.GRUCell(size)
        print('single_cell')
        if use_lstm:
            single_cell = tf.contrib.rnn.LSTMCell(size)
        cell = single_cell
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.5)
        if num_layers > 1:
            cell = tf.contrib.rnn.MultiRNNCell([single_cell] * num_layers)
        # tf.contrib.seq2seq
        # The seq2seq function: we use embedding for the input and attention.
        def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
            return embedding_attention_seq2seq(
                  encoder_inputs, decoder_inputs, cell,
                  num_encoder_symbols=source_vocab_size,
                  num_decoder_symbols=target_vocab_size,
                  embedding_size=size,
                  output_projection=output_projection,
                  feed_previous=do_decode)

        # Feeds for inputs.
        self.encoder_inputs = []
        self.decoder_inputs = []
        self.target_weights = []
        for i in xrange(self.buckets[-1][0]):  # Last bucket is the biggest one.
            self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="encoder{0}".format(i)))
        for i in xrange(self.buckets[-1][1] + 1):
            self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="decoder{0}".format(i)))
            self.target_weights.append(tf.placeholder(tf.float32, shape=[None],
                                                name="weight{0}".format(i)))
        self.reward = [tf.placeholder(tf.float32, name="reward_%i" % i) for i in range(len(self.buckets))] # ADD
        
        
        # Our targets are decoder inputs shifted by one.
        targets = [self.decoder_inputs[i + 1] for i in xrange(len(self.decoder_inputs) - 1)]

        self.outputs, self.losses,self.encoder_state  = model_with_buckets(
                  self.encoder_inputs, self.decoder_inputs, targets,
                  self.target_weights, self.buckets, source_vocab_size, self.batch_size,
                  lambda x, y: seq2seq_f(x, y, tf.where(self.forward_only, True, False)),
                  output_projection=output_projection,
                  softmax_loss_function=softmax_loss_function)
        
        for b in xrange(len(self.buckets)):
            self.outputs[b] = [
                tf.cond(
                    self.forward_only,
                    lambda: tf.matmul(output, output_projection[0]) + output_projection[1],
                    lambda: output
                )
                for output in self.outputs[b]
            ]
        
        if not forward_only:
            with tf.name_scope("gradient_descent"):
                self.gradient_norms = []
                self.updates = []
                self.aj_losses = []
                self.gen_params = [p for p in tf.trainable_variables() if name_scope in p.name]
                opt = tf.train.AdamOptimizer()
                for b in xrange(len(self.buckets)):
                    R =  tf.subtract(self.reward[b], self.reward_bias)
                    adjusted_loss = tf.cond(self.up_reward,
                                              lambda:tf.multiply(self.losses[b], self.reward[b]),
                                              lambda: self.losses[b])

                    self.aj_losses.append(adjusted_loss)
                    gradients = tf.gradients(adjusted_loss, self.gen_params)
                    clipped_gradients, norm = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
                    self.gradient_norms.append(norm)
                    self.updates.append(opt.apply_gradients(
                        zip(clipped_gradients, self.gen_params), global_step=self.global_step))

        self.gen_variables = [k for k in tf.global_variables() if name_scope in k.name]
        self.saver = tf.train.Saver(tf.global_variables())

    def step(self, session, encoder_inputs, decoder_inputs, target_weights,
           bucket_id, forward_only=True, reward=1, mc_search=False, up_reward=False, debug=True):
        #print("model.step")
        # Check if the sizes match.
        encoder_size, decoder_size = self.buckets[bucket_id]
        if len(encoder_inputs) != encoder_size:
            raise ValueError("Encoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(encoder_inputs), encoder_size))
        if len(decoder_inputs) != decoder_size:
            raise ValueError("Decoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(decoder_inputs), decoder_size))
        if len(target_weights) != decoder_size:
            raise ValueError("Weights length must be equal to the one in bucket,"
                       " %d != %d." % (len(target_weights), decoder_size))
        #print("input feed")
        # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
        input_feed = {self.forward_only.name: forward_only,
                      self.up_reward.name:  up_reward,
                      self.mc_search.name: mc_search
                     }
        for l in xrange(len(self.buckets)):
            input_feed[self.reward[l].name] = reward
        for l in xrange(encoder_size):
            input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
        for l in xrange(decoder_size):
            input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
            input_feed[self.target_weights[l].name] = target_weights[l]
        #print("last_target")
        # Since our targets are decoder inputs shifted by one, we need one more.
        last_target = self.decoder_inputs[decoder_size].name
        input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)

        # Output feed: depends on whether we do a backward step or not.
        if not forward_only:
            #print("not forward_only")
            output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                     self.gradient_norms[bucket_id],  # Gradient norm.
                     self.losses[bucket_id]]  # Loss for this batch.
        else:
            #print("forward_only")
            output_feed = [self.losses[bucket_id]]  # Loss for this batch.
            for l in xrange(decoder_size):  # Output logits.
                output_feed.append(self.outputs[bucket_id][l])

        outputs = session.run(output_feed, input_feed)
        if not forward_only:
              return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
        else:
              return None, outputs[0], outputs[1:]  # No gradient norm, loss, outputs.

    def get_batch(self, data, bucket_id):

        encoder_size, decoder_size = self.buckets[bucket_id]
        encoder_inputs, decoder_inputs = [], []

        for _ in xrange(self.batch_size):
            encoder_input, decoder_input = random.choice(data[bucket_id])

            # Encoder inputs are padded and then reversed.
            encoder_pad = [PAD_ID] * (encoder_size - len(encoder_input))
            encoder_inputs.append(list(reversed(encoder_input + encoder_pad)))

            # Decoder inputs get an extra "GO" symbol, and are padded then.
            decoder_pad_size = decoder_size - len(decoder_input) - 1
            decoder_inputs.append([GO_ID] + decoder_input +
                            [PAD_ID] * decoder_pad_size)

        # Now we create batch-major vectors from the data selected above.
        batch_encoder_inputs, batch_decoder_inputs, batch_weights = [], [], []

        # Batch encoder inputs are just re-indexed encoder_inputs.
        for length_idx in xrange(encoder_size):
            batch_encoder_inputs.append(
                np.array([encoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

        # Batch decoder inputs are re-indexed decoder_inputs, we create weights.
        for length_idx in xrange(decoder_size):
            batch_decoder_inputs.append(
                np.array([decoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

            # Create target_weights to be 0 for targets that are padding.
            batch_weight = np.ones(self.batch_size, dtype=np.float32)
            for batch_idx in xrange(self.batch_size):
                # We set weight to 0 if the corresponding target is a PAD symbol.
                # The corresponding target is decoder_input shifted by 1 forward.
                if length_idx < decoder_size - 1:
                    target = decoder_inputs[batch_idx][length_idx + 1]
                if length_idx == decoder_size - 1 or target == PAD_ID:
                    batch_weight[batch_idx] = 0.0
            batch_weights.append(batch_weight)
        return batch_encoder_inputs, batch_decoder_inputs, batch_weights, encoder_inputs,decoder_inputs

In [14]:
'''with open('first_gan.csv', 'w') as datacsv:
    writer = csv.writer(datacsv,dialect=("excel"))
    writer.writerow(['question_id','question','target_answer'])'''

'with open(\'first_gan.csv\', \'w\') as datacsv:\n    writer = csv.writer(datacsv,dialect=("excel"))\n    writer.writerow([\'question_id\',\'question\',\'target_answer\'])'

In [15]:
def read_data(config):

    train_data_set = [[] for _ in config.buckets]
    dev_data_set = [[] for _ in config.buckets]

    s2s_data = pd.read_csv('second_gan.csv')
    train_data = s2s_data[0:25000]
    dev_data = s2s_data[25000:]
    
    train_source = train_data['candidate_answer']
    train_target = train_data['target_answer']
    
    dev_source = dev_data['candidate_answer']
    dev_target = dev_data['target_answer']
    
    vocabulary = json.load(open('vocabulary_QA.json','r'))
    
    for m in range(0,len(train_source)):
        if m % 10000 == 0:
            print("  reading train data line %d" % m)
            sys.stdout.flush()
            
        source_sen = train_source[m]
        target_sen = train_target[m]
            
        source_word = []
        target_word = []
        
        try:
            s_w = source_sen.split(' ')
        except:
            s_w = ['float']
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            source_word.append(s_w[i])
            
        s_w = target_sen.split(' ')
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            target_word.append(s_w[i])            
            
        source_ids = []
        target_ids = []
            
        # convert to number
        
        for w in source_word:
            try:
                source_ids.append(vocabulary[w])
            except:
                source_ids.append(3)# 3 is UNK token
                
        # add start tag to target sentence
        #target_ids.append(1) # 1 is _GO: start token
  
        for w in target_word:
            try:
                target_ids.append(vocabulary[w])
            except:
                target_ids.append(3)# 3 is UNK token

        # add finish tag to target sentence. 
        target_ids.append(EOS_ID) # EOS_ID is end token
        
        
        # bucket data
        #config.buckets:  [bucket_id, (source_size, target_size)]
        for bucket_id, (source_size, target_size) in enumerate(config.buckets): #[bucket_id, (source_size, target_size)]
            if len(source_ids) < source_size and len(target_ids) < target_size:
                train_data_set[bucket_id].append([source_ids, target_ids])
                break
                 
    for m in range(25000,25000+len(dev_source)):
        if m % 10000 == 0:
            print("  reading dev data line %d" % m)
            sys.stdout.flush()
            
        source_sen = dev_source[m]
        target_sen = dev_target[m]
 
        source_word = []
        target_word = []
        
        try:
            s_w = source_sen.split(' ')
        except:
            s_w = ['float']
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            source_word.append(s_w[i])
            
        s_w = target_sen.split(' ')
        for i in range(0,len(s_w)):
            s_w[i] = s_w[i].replace('.','')
            s_w[i] = s_w[i].replace(',','')
            s_w[i] = s_w[i].replace('?','')
            s_w[i] = s_w[i].replace('(','')
            s_w[i] = s_w[i].replace(')','')
            s_w[i] = s_w[i].replace('!','')
            s_w[i] = s_w[i].lower()
            target_word.append(s_w[i])    
            
        source_ids = []
        target_ids = []
            
        # convert to number
        for w in source_word:
            try:
                source_ids.append(vocabulary[w])
            except:
                source_ids.append(3)
                
        #target_ids.append(1) # 1 is _GO: start token
        for w in target_word:
            try:
                target_ids.append(vocabulary[w])
            except:
                target_ids.append(3)

        # add finish tag to target sentence. 
        target_ids.append(EOS_ID) # End token
                
        # bucket data
        #config.buckets:  [bucket_id, (source_size, target_size)]
        for bucket_id, (source_size, target_size) in enumerate(config.buckets): #[bucket_id, (source_size, target_size)]
            if len(source_ids) < source_size and len(target_ids) < target_size:
                dev_data_set[bucket_id].append([source_ids, target_ids])
                break
                                
    return vocabulary,train_data_set,dev_data_set
    #,train_query,dev_query




In [16]:
def prepare_data(gen_config):
    vocab, train_set,dev_set = read_data(gen_config)
    rev_vocab = {v: k for k, v in vocab.items()}
    return vocab, rev_vocab, dev_set, train_set
    #,train_query,dev_query

In [17]:
def create_model(session, gen_config, forward_only,name_scope, initializer=None):
    """Create translation model and initialize or load parameters in session."""
    with tf.variable_scope(name_or_scope=name_scope, initializer=initializer):
        model = Seq2SeqModel(gen_config, name_scope=name_scope, forward_only=forward_only)
        
        # checkpoint
        gen_ckpt_dir = os.path.abspath(os.path.join(gen_config.train_dir, "checkpoints"))
        # os.path.abspath: 返回path规范化的绝对路径。 
        ckpt = tf.train.get_checkpoint_state(gen_ckpt_dir)
        
        if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
            print("Reading Gen model parameters from %s" % ckpt.model_checkpoint_path)
            model.saver.restore(session, ckpt.model_checkpoint_path)
            #return model
        else:
            print("Created Gen model with fresh parameters.")
            gen_global_variables = [gv for gv in tf.global_variables() if name_scope in gv.name]
            session.run(tf.variables_initializer(gen_global_variables))
        return model

## pre train generator

In [18]:
def train(gen_config):
    
    vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
    
    for b_set in train_set:
        print("bucket_set: ", len(b_set))

    with tf.Session() as sess:
    #with tf.device("/gpu:1"):
        # Create model.
        print("Creating %d layers of %d units." % (gen_config.num_layers, gen_config.emb_dim))
        model = create_model(sess, gen_config, forward_only=False,name_scope=gen_config.name_model)
        
        #size of each bucket in the train_dataset.
        train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
        # add them to get total train dataset size
        train_total_size = float(sum(train_bucket_sizes))
        # 0~1 --> id of bucket data.
        train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                               for i in xrange(len(train_bucket_sizes))]

        # This is the training loop.
        step_time, loss = 0.0, 0.0
        current_step = 0
        previous_losses = []
        gen_loss_summary = tf.Summary()
        gen_writer = tf.summary.FileWriter(gen_config.tensorboard_dir, sess.graph)
        print("training.......")
        while True:
            # Choose a bucket.
            # Choose a bucket according to data distribution. We pick a random number
            # in [0, 1] and use the corresponding interval in train_buckets_scale.
            random_number_01 = np.random.random_sample()
            bucket_id = min([i for i in xrange(len(train_buckets_scale)) if train_buckets_scale[i] > random_number_01])

            
            # Get a batch and make a step.
            start_time = time.time()
            encoder_inputs, decoder_inputs, target_weights,_,_ = model.get_batch(train_set, bucket_id)
            # model.get_batch 和 model.step 都是seq2seq里的函数。
            _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, forward_only=False)

            step_time += (time.time() - start_time) / gen_config.steps_per_checkpoint
            loss += step_loss / gen_config.steps_per_checkpoint
            current_step += 1

            
            
            if current_step % gen_config.steps_per_checkpoint == 0:
                bucket_value = gen_loss_summary.value.add()
                bucket_value.tag = gen_config.name_loss
                bucket_value.simple_value = float(loss)
                gen_writer.add_summary(gen_loss_summary, int(model.global_step.eval()))

                # Print statistics for the previous epoch.
                perplexity = math.exp(loss) if loss < 300 else float('inf')
                print ("global step %d learning rate %.4f step-time %.2f perplexity "
                       "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                                 step_time, perplexity))

                # Decrease learning rate if no improvement was seen over last 3 times.
                if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
                    sess.run(model.learning_rate_decay_op)
                previous_losses.append(loss)
                
                if current_step % (gen_config.steps_per_checkpoint * 3) == 0:
                    print("current_step: %d, save model" %(current_step))
                    gen_ckpt_dir = os.path.abspath(os.path.join(gen_config.train_dir, "checkpoints"))
                    if not os.path.exists(gen_ckpt_dir):
                        #os.makedirs() 方法用于递归创建目录
                        os.makedirs(gen_ckpt_dir)
                    checkpoint_path = os.path.join(gen_ckpt_dir, "chitchat.model")
                    model.saver.save(sess, checkpoint_path, global_step=model.global_step)

                step_time, loss = 0.0, 0.0
                sys.stdout.flush()




In [24]:
class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 64
    emb_dim = 256
    num_layers = 4
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_d/"
    tensorboard_dir = "./d_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(5,15),(10,20),(15,25),(30,30)]
    buckets_concat = [(5,15),(10,20),(15,25),(30,30)]

In [25]:
train(gen_config)

  reading train data line 0
  reading train data line 10000
  reading train data line 20000
bucket_set:  0
bucket_set:  673
bucket_set:  5521
bucket_set:  8373
Creating 4 layers of 256 units.
single_cell
Reading Gen model parameters from /Users/zhangyiman/final project/gen_data_d/checkpoints/chitchat.model-3600
training.......
global step 3700 learning rate 0.5000 step-time 3.95 perplexity 28.32
global step 3800 learning rate 0.5000 step-time 3.96 perplexity 23.63
global step 3900 learning rate 0.5000 step-time 3.98 perplexity 21.50
current_step: 300, save model
global step 4000 learning rate 0.5000 step-time 3.94 perplexity 19.89
global step 4100 learning rate 0.5000 step-time 4.02 perplexity 18.65
global step 4200 learning rate 0.5000 step-time 4.06 perplexity 17.02
current_step: 600, save model
global step 4300 learning rate 0.5000 step-time 3.61 perplexity 15.80
global step 4400 learning rate 0.5000 step-time 3.71 perplexity 14.42
global step 4500 learning rate 0.5000 step-time 3.9

KeyboardInterrupt: 

## seq2seq decode

In [19]:
class gen_config(object):
    beam_size = 10
    learning_rate = 0.5
    learning_rate_decay_factor = 0.99
    max_gradient_norm = 2.0
    batch_size = 1
    emb_dim = 256
    num_layers = 4
    vocab_size = 250000
    name_model = "st_model"
    train_dir = "./gen_data_d/"
    tensorboard_dir = "./d_tensorboard/gen_log/"
    name_loss = "gen_loss"
    teacher_loss = "teacher_loss"
    reward_name = "reward"
    max_train_data_size = 0
    steps_per_checkpoint = 100
    buckets = [(5,15),(10,20),(15,25),(30,30)]
    buckets_concat = [(5,15),(10,20),(15,25),(30,30)]

In [20]:
def decode(decode_num_step):
    
    # Load vocabularies.
    vocab, rev_vocab, dev_set, train_set = prepare_data(gen_config)
        
        
    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(gen_config.buckets))]
    train_total_size = float(sum(train_bucket_sizes))
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]
    
    input_document = []
    target_summary = []
    generated_summary = []
    with tf.Session() as sess:
        # Create model and load parameters.
        model = create_model(sess, gen_config, forward_only=True, name_scope=gen_config.name_model)

        num_step = 0
        while num_step < decode_num_step:
            print("generating num_step: ", num_step)
            random_number_01 = np.random.random_sample()
            bucket_id = min([i for i in xrange(len(train_buckets_scale))
                             if train_buckets_scale[i] > random_number_01])
            # Get a 1-element batch to feed the sentence to the model.
            encoder_inputs, decoder_inputs, target_weights, _encoder_inputs, _decoder_inputs = model.get_batch(
                  train_set, bucket_id)#get_batch(train_set,bucket_id)
            # Get output logits for the sentence.
            _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,target_weights, bucket_id, True)
            # This is a greedy decoder - outputs are just argmaxes of output_logits.
            outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
            #print('batch_encoder_input')
            #print(" ".join([str(rev_vocab[an]) for an in encoder_inputs]))
            # If there is an EOS symbol in outputs, cut them at that point.
            if EOS_ID in outputs:
                outputs = outputs[:outputs.index(EOS_ID)]
            # Print out French sentence corresponding to outputs.
            summary_g = " ".join([tf.compat.as_str(rev_vocab[output]) for output in outputs])
            print(summary_g)
            generated_summary.append(summary_g)
            
            for query, answer,outputs in zip(_encoder_inputs, _decoder_inputs,outputs):

                answer_str = " ".join([str(rev_vocab[an]) for an in answer])
                answer_str = answer_str.replace('_GO ','')
                answer_str = answer_str.replace(' _EOS','')
                answer_str = answer_str.replace(' _PAD','')
                print(answer_str)
                target_summary.append(answer_str)
                
                query_str = " ".join([str(rev_vocab[qu]) for qu in query])
                query_str = query_str.replace(' _PAD','')
                #print(query_str)
                document_str = query_str.split(' ')
                i = len(document_str)-1
                d_str = ''
                while i>0:
                    d_str = d_str + document_str[i]
                    d_str = d_str + ' '
                    i = i-1
                #print(d_str)
                input_document.append(d_str)
            num_step +=1
    return input_document, target_summary, generated_summary

In [21]:
list1,list2,list3 = decode(50)

  reading train data line 0
  reading train data line 10000
  reading train data line 20000
single_cell
Reading Gen model parameters from /Users/zhangyiman/final project/gen_data_d/checkpoints/chitchat.model-5700
generating num_step:  0
  stands and is used _UNK _UNK 
  hours ahead _UNK is hours ahead and _UNK is hours ahead 
generating num_step:  1
  open on on 
  click on modems tab 
generating num_step:  2
  in asking it the same right the answer 
  by placing your question in the proper forum and not the _UNK one 
generating num_step:  3
  women are more you need to need more information and free 
  those are just sitting around online you need to contact the texas vital statistics department 
generating num_step:  4
  yahoo your blocker but either when will erase yahoo on yahoo 
  edit my info communications check mark allow yahoo contact via messenger 
generating num_step:  5
  _UNK of god _UNK _UNK _UNK the _UNK _UNK the _UNK 
  _UNK of egypt during egypt _UNK age the _UNK was t

In [ ]:
#Flag = True # Flag = true: write a csv file Flag = False, don't write
#Flag = False
#decode_num_step = 50000
#generated_data_for_D(Flag,decode_num_step)